#### Quantization 종류
- 기존의 high precision(일반적으로 fp32) Neural network의 weights와 activation을 더 적은 bit(low precision)으로 변환하는 것
- Quantized Matrix Multiplication, Activation, Layer fusion,,,

#### ex) quantized code
- quantized_cod=uint8(((original-min)/(max-min))*255)

- reconstructured=((quantized_code/255.0)*(max-min)+min

#### Quantization approach 구분
- Post Training Quantization(PTQ): 학습 후에 quantization parameter(scale, shift)를 결정
- Quantization Aware Training(QAT):학습 과정에 quantization을 emulate함으로써, quantization으로 발생하는 성능하락을 완화함

#### PTQ 기법 정리
- Dynamic range quantization(weight only quantization): weight만 quantize됨(8bit), inference 시에는 floating-point로 변환되어 수행
- Full integer quantization(weight and activation quantization): weight와 더불어 모델의 입력 데이터, activation(중간 레이어의 output)들 또한 quantize, Post Training Quantization, Quantization-Aware Training이 여기에 속함
- Float16 quantization: fp32의 데이터 타입의 weight를 fp16으로 quantize

참고: Pytorch 구분
- Dynamic range quantization(Dynamic quantization)을 별도의 범주로 구분
- Post Training Quantization은 Static quantization이라 칭함
- Dynamic quantization의 경우, model 수행 시간이 weights를 load하는 것이 실제 matrix multiplication보다 더 오래 걸리는 LSTM, Transformer 기반의 모델에 효과적이라는 언급이 있음

1) Dynamic range quantization(weight only quantization)
- 네트워크의 Weight만 quantize됨(8bit)
- Pros:
  - 별도의 calibration(validation)데이터가 필요하지 않음
  - 모델의 용량 축소(8bit 기준 1/4)
- Cons:
 - 실제 연산은 floating point로 수행됨

#### 2) Full integer quantization(weight and activation quantization)
- Weight와 더불어 모델의 입력 데이터, activation(중간 레이어의 output)들도 quantize됨
- Pros:
  - 모델의 용량 축소(8bit 기준 1/4)
  - 더 적은 메모리 사용량, cache 재사용성 증가
  - 빠른 연산(fixed point 8bit 연산을 지원하는 경우)
- Cons:
 - Activation의 parameter를 결정하기 위하여 calibration 데이터가 필요함
  (주로 training 데이터에서 사용, 약 100개의 데이터)

2) Full integer quantization; TensorRT(NVIDIA) calibration 예시
- Calibration?; 성능 저하를 최소로 하는 threshold 찾기
- Minimize information loss로 관점으로 접근
- 각 네트워크, 각 레이어 마다 activation value의 range, distribution가 다르다(x축: value, y축: normalized histogram counts)


#### 3) Float16 quantization
- Float 32 모델을 float 16모델로 변환
- Pros:
 - 모델의 용량 축소(1/2)
 - 적은 성능 저하
 - GPU 상에서 빠른 연산(대체로 fp32를 상회)
- Cons:
 - CPU 상에서는 fixed point 연산만큼의 속도 향상이 있지는 않음

#### 1.3 Quantization Aware Training(QAT)
- 학습 과정에서 quantization을 emulate하여(fake quantization), inference시에 발생하는 quantization error를 training 시점에 반영가능하도록 함
- 보통은 일반적인 방법으로 학습을 진행하고, finetuning으로 QAT 적용
- 학습 과정에 emulate된 quantization 파라미터를 inference에도 사용
- PTQ 대비 성능 하락 폭이 적음

- 학습 과정
 - 학습 과정 중 quantization을 적용하고, 다시 floating point로 변환함(backprop을 계산하기 위함)
 - In-out에 대한 gradient를 linear로 가정(straing-through estimator)함으로써 네트워크 학습을 수행

#### PyTorch에서 요구하는 준비사항(PTQ, QAT)
1. activation이 어디서 quantized되고 de-quantized되는지 구체화해야 한다. (QuantStub, DeQuantStub module을 사용해야 한다.)
2. torch.nn.quantized.FloatFunctional을 사용하여 quantization을 위해 특별한 처리가 필요한 텐서 연산을 module로 랩핑. output quantization parameter를 결정하기 위해 특별한 처리가 필요한 add와 cat과 같은 연산이 그 예시.
3. Fuse modules: operation/module을 single module로 결합하여 더 높은 정확도와 성능을 얻을 수 있다. 이는 융합할 모듈 리스트를 가져오는 torch.quantization.fuse_modules() API를 사용하여 수행된다. 현재 다음과 같은 fusion을 지원: [Conv, Relu], [Conv,BatchNorm],[Conv,BatchNorm,Relu],[Linear,Relu]


In [1]:
# QuantStub, DeQuantStub
import torch

# quantize 가능한 단순한 floating point Model을 정의
class Model(torch.nn.Module):
  def __init__(self):
    super(Model,self).__init__()

    # QuantStub을 이용해 floating point에서 quantized tensor로 변환한다.
    self.quant=torch.quantization.QuantStub()

    # Conv2d (in channel, out channel, kernel size)
    self.conv=torch.nn.Conv2d(3,3,1)
    self.relu=torch.nn.ReLU()
    self.flatten=torch.nn.Flatten()

    # Image size: 32 x 32
    self.linear=torch.nn.Linear(3*32*32,10)

    # DeQuantStub을 이용해 quantized tensor에서 floating point로 변환한다.
    self.dequant=torch.quatnization.DeQuantStub()

  def forward(self,x):
    x=self.quant(x)
    x=self.conv(x)
    x=self.relu(x)

    x=self.flatten(x)
    x=self.linear(x)
    x=self.dequant(x)

    return x

In [4]:
import torchvision
model_quantized=torchvision.models.quantization.mobilenet_v2(pretrained=True,quantize=True)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_QuantizedWeights.IMAGENET1K_QNNPACK_V1`. You can also use `weights=MobileNet_V2_QuantizedWeights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torch/ao/quantization/utils.py:339: UserWarning: must run observer before calling calculate_qparams. Returning default values.
  warnings.warn(
Downloading: "https://download.pytorch.org/models/quantized/mobilenet_v2_qnnpack_37f702c5.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2_qnnpack_37f70

In [5]:
model=torchvision.models.mobilenet_v2(pretrained=True)

import os
import torch

def print_model_size(mdl):
  torch.save(mdl.state_dict(),"tmp.pt")
  print("%.2f MB" %(os.path.getsize("tmp.pt")/1e6))
  os.remove('tmp.pt')

print_model_size(model)
print_model_size(model_quantized)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 60.3MB/s]


14.24 MB
3.62 MB


In [6]:
# 학습 후 동적 양자화(Post Training Dynamic Quantization)

model_dynamic_quantized=torch.quantization.quantize_dynamic(model,qconfig_spec={torch.nn.Linear},dtype=torch.qint8)

In [8]:
# 학습 후 정적 양자화(Post Training Static Quantization)

backend="qnnpack"
model.qconfig = torch.quantization.get_default_qconfig(backend)
torch.backends.quantized.engine = backend
model_static_quantized = torch.quantization.prepare(model, inplace=False)
model_static_quantized = torch.quantization.convert(model_static_quantized, inplace=False)

/usr/local/lib/python3.10/dist-packages/torch/ao/quantization/observer.py:1263: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  warnings.warn(


In [10]:
model.qconfig = torch.quantization.get_default_qat_qconfig(backend)
model_qat = torch.quantization.prepare_qat(model, inplace=False)
# 양자화를 고려한 학습이 여기서 진행됩니다.
model_qat = torch.quantization.convert(model_qat.eval(), inplace=False)

/usr/local/lib/python3.10/dist-packages/torch/ao/quantization/utils.py:339: UserWarning: must run observer before calling calculate_qparams. Returning default values.
  warnings.warn(


In [15]:
# 아래 Quantization Aware Training에서도 이 클래스 사용
class CustomModel(torch.nn.Module) :
  def __init__(self) :
    super(CustomModel,self).__init__()
    self.Quantizer = torch.quantization.QuantStub()

    self.dequantizer = torch.quantization.DeQuantStub()

  def forward(self, x) :
    x = self.Quantizer(x)
    x = self.conv(x)
    x = self.batchnorm(x)
    x = self.relu(x)		# conv, batchnorm, relu fuse할 예정
    x = self.dequantizer(x)
    return x
model_float32 = CustomModel()

In [21]:
model_float32.eval()		# 평가모드
model_float32.qconfig = torch.quantization.get_default_qconfig('fbgemm')
#model_float32 = torch.quantization.fuse_modules(model_float32,  [['conv', 'batchnorm', 'relu']])
#model_float32_prepared = torch.quantization.prepare(model_float32_fused)

#model_float32_prepared(input_float32)	# 실제 dataset으로 파라미터 교정
#model_int8 = torch.quantization.convert(model_float32_prepared)
#res = model_int8(input_float32)		# 추론